# 대화 이력 관리를 위한 메모리 구현 (Chat History)

### **학습 목표**
1. LangChain의 다양한 메모리 관리 방식 이해 (레거시 vs 최신)
2. RunnableWithMessageHistory를 활용한 대화 이력 관리 (레거시 방식)
3. LangChain 1.0의 `create_agent` + Checkpointer를 활용한 메모리 구현 (권장)
4. 메시지 트리밍과 대화 요약을 통한 컨텍스트 윈도우 관리

---

## 환경 설정 및 준비

### 사전 준비

**필수 환경 변수:**
`.env` 파일에 다음 내용을 추가하세요:
```
OPENAI_API_KEY=sk-...
```

**필수 패키지:**
```bash
uv add langchain langchain-openai langgraph langgraph-checkpoint-sqlite
```

**참고:**
- LangChain 1.0부터 `create_agent`가 에이전트 생성의 표준 방식입니다
- `create_agent`는 내부적으로 LangGraph 기반으로 동작합니다

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from pprint import pprint

`(3) LLM 설정`

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',      # 사용할 모델
    temperature=0.3,            # 응답의 창의성 조절 (0~1, 낮을수록 일관적)
    top_p=0.9,                  # 토큰 샘플링 확률 임계값 (0~1)
)

## 기본 개념: 단기 메모리 vs 장기 메모리

- **단기 메모리 (Short-term Memory)**
    - **Thread 기반 대화 관리**: 각 대화 세션(thread)별로 메시지 히스토리를 유지
    - **Checkpointer**를 통해 에이전트 실행 시점마다 상태를 저장
    - 같은 `thread_id`를 사용하면 이전 대화 내용을 기억

- **장기 메모리 (Long-term Memory)**
    - **Store**를 통해 thread를 넘어서 정보를 공유
    - 사용자별 선호도, 프로필 정보 등을 저장
    - 여러 대화 세션에서 공통으로 접근 가능

- **LangChain 1.0 권장 패턴**
    - `create_agent`가 에이전트 생성의 표준 방식
    - 내부적으로 LangGraph 기반으로 동작
    - `checkpointer` 파라미터로 메모리 관리 자동화

---

## 1. 메시지 전달 방식 (Message Passing)

* 메시지 전달 방식은 LangChain에서 가장 기본적인 메모리 구현 방법으로, 이전 대화 기록(chat history)을 체인에 직접 전달하여 문맥을 유지하는 방식입니다.

* 이 방식은 SystemMessage(시스템 지시사항), HumanMessage(사용자 입력), AIMessage(AI 응답) 등 다양한 유형의 메시지를 ChatPromptTemplate을 통해 구조화하며, MessagesPlaceholder를 사용하여 이전 대화 내용을 포함시킵니다.

* 챗봇의 기본적인 메모리 시스템을 구현하는데 사용되며, 이를 통해 AI는 이전 대화 맥락을 이해하고 그에 맞는 적절한 응답을 생성할 수 있습니다.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ChatPromptTemplate를 사용하여 챗봇의 초기 메시지를 정의
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="messages"),    # 메시지 목록을 동적으로 삽입하는 부분
])

# ChatPromptTemplate에 삽입할 메시지 목록을 정의
messages = [
        HumanMessage(content="안녕하세요. 제 이름은 홍길동입니다."),
        AIMessage(content="안녕하세요! 어떻게 도와드릴까요?"),
]    

# ChatPromptTemplate에 삽입할 메시지 목록을 업데이트하고 출력 
pprint(prompt.format(messages=messages))

In [ ]:
# 대화형 체인을 정의 (prompt -> llm)
chain = prompt | llm

# 기본적인 메시지 전달: 이전 메시지 목록에 새로운 메시지를 추가해서 전달
response = chain.invoke({
    "messages": messages + [HumanMessage(content="제 이름을 기억하나요?")] # 이전 메시지를 기억하는지 확인하는 질문 메시지 추가
})

pprint(response)

---

## 2. RunnableWithMessageHistory (레거시 방식)

* `RunnableWithMessageHistory`는 LangChain에서 대화 기록을 관리하는 고급 기능으로, 체인의 실행 과정에서 메시지 기록을 자동으로 저장하고 검색할 수 있게 해주는 래퍼(wrapper) 클래스입니다.

* 이 기능은 대화 세션별로 독립적인 기록을 유지할 수 있게 해주며, `ConfigurableField`를 통해 메모리 구성을 유연하게 조정할 수 있습니다.

* 구현 시에는 `get_session_history` 콜백을 통해 세션ID별로 메시지 기록을 관리하며, 이를 통해 각 대화의 컨텍스트를 정확하게 유지할 수 있습니다.

    | 저장소 유형 | 클래스 | 용도 |
    |------------|--------|------|
    | 메모리 기반 | `InMemoryHistory` (커스텀) | 개발/테스트용, 휘발성 |
    | SQLite | `SQLiteChatMessageHistory` (커스텀) | 로컬 영구 저장 |

### 2.1 InMemoryHistory (메모리 기반) 구현

* `InMemoryHistory` 클래스는 대화 이력의 기본 구조를 제공하며, `BaseChatMessageHistory`와 `BaseModel`을 상속받아 메시지를 메모리에서 효율적으로 관리합니다.

* `store` 변수는 전역 딕셔너리로 구현되어 세션별 대화 이력을 구분하여 저장합니다. `session_id`를 키로 사용하여 각 세션의 `InMemoryHistory` 객체에 빠르게 접근할 수 있습니다.

* `get_session_history` 함수는 세션 관리의 진입점 역할을 하며, 존재하지 않는 세션에 대해 자동으로 새로운 `InMemoryHistory` 객체를 생성하는 팩토리 패턴을 구현합니다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from typing import List

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """메모리 기반 대화 히스토리 저장소"""
    messages: List[BaseMessage] = Field(default_factory=list)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """메시지 추가"""
        self.messages.extend(messages)
    
    def clear(self) -> None:
        """히스토리 초기화"""
        self.messages = []

# 세션 저장소 (전역 딕셔너리)
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """세션 ID에 해당하는 히스토리 반환 (없으면 새로 생성)"""
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

print("InMemoryHistory 클래스가 정의되었습니다.")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# 프롬프트 템플릿 설정
prompt_legacy = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요."),
    MessagesPlaceholder(variable_name="history"), 
    ("human", "{input}")
])

# 프롬프트와 llm을 연결하여 체인 생성
chain_legacy = prompt_legacy | llm

# 히스토리 관리 추가  
chain_with_history = RunnableWithMessageHistory(
    chain_legacy,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 지정된 세션 ID(tourist_legacy_1)를 사용하여 체인 실행
response = chain_with_history.invoke(
    {"input": "서울에서 가볼만한 곳을 추천해주세요."},
    config={"configurable": {"session_id": "tourist_legacy_1"}}
)

print("여행 가이드 답변:")
print(response.content)

In [ ]:
# 대화 히스토리 출력
history = get_session_history("tourist_legacy_1")
pprint(history.messages)

In [ ]:
# 이전 대화 내용을 기반으로 후속 질문
response = chain_with_history.invoke(
    {"input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"},
    config={"configurable": {"session_id": "tourist_legacy_1"}}
)

print("여행 가이드 답변:")
print(response.content)

### 2.2 SQLiteChatMessageHistory (영구 저장소)

* SQLite 통합 구현을 위해서는 먼저 메시지를 저장할 데이터베이스 테이블 구조를 정의하고, `BaseChatMessageHistory`를 상속받아 메시지 저장/조회 로직을 구현해야 합니다.

* 세션 ID를 기준으로 대화 내용을 구분하여 관리하며, 메시지의 타입(Human/AI), 내용, 메타데이터, 타임스탬프 등의 정보를 체계적으로 저장합니다.

* 프로세스가 재시작되어도 대화 내용이 유지되어, 프로덕션 환경에서 사용하기 적합합니다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import sqlite3
from typing import List
import json

class SQLiteChatMessageHistory(BaseChatMessageHistory):
    """
    SQLite 데이터베이스를 사용하여 챗봇 대화 히스토리를 저장하는 클래스

    Attributes:
        session_id (str): 세션 ID
        db_path (str): SQLite 데이터베이스 파일 경로
    """
    def __init__(self, session_id: str, db_path: str = "chat_history_legacy.db"):
        self.session_id = session_id
        self.db_path = db_path
        self._create_tables()
    
    def _create_tables(self):
        """데이터베이스 테이블 생성"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS messages (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                session_id TEXT,
                message_type TEXT,
                content TEXT,
                metadata TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        conn.commit()
        conn.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """단일 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO messages (session_id, message_type, content, metadata)
            VALUES (?, ?, ?, ?)
        """, (
            self.session_id,
            message.__class__.__name__,
            message.content,
            json.dumps(message.additional_kwargs)
        ))
        
        conn.commit()
        conn.close()
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """여러 메시지 추가"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        for message in messages:
            cursor.execute("""
                INSERT INTO messages (session_id, message_type, content, metadata)
                VALUES (?, ?, ?, ?)
            """, (
                self.session_id,
                message.__class__.__name__,
                message.content,
                json.dumps(message.additional_kwargs)
            ))
        
        conn.commit()
        conn.close()
    
    def clear(self) -> None:
        """세션의 모든 메시지 삭제"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            DELETE FROM messages WHERE session_id = ?
        """, (self.session_id,))
        
        conn.commit()
        conn.close()
    
    @property
    def messages(self) -> List[BaseMessage]:
        """저장된 메시지 조회"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute("""
            SELECT message_type, content, metadata
            FROM messages 
            WHERE session_id = ?
            ORDER BY created_at
        """, (self.session_id,))
        
        messages = []
        for row in cursor.fetchall():
            message_type, content, metadata = row
            if message_type == "HumanMessage":
                message = HumanMessage(content=content)
            else:
                message = AIMessage(content=content)
            
            if metadata:
                message.additional_kwargs = json.loads(metadata)
            
            messages.append(message)
        
        conn.close()
        return messages

# 세션 ID로 히스토리 가져오기
def get_sqlite_history(session_id: str) -> BaseChatMessageHistory:
    return SQLiteChatMessageHistory(session_id=session_id)

print("SQLiteChatMessageHistory 클래스가 정의되었습니다.")

In [ ]:
# SQLite 히스토리를 사용하는 RunnableWithMessageHistory 체인 구성
chain_with_sqlite = RunnableWithMessageHistory(
    chain_legacy,
    get_sqlite_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 테스트 대화
response = chain_with_sqlite.invoke(
    {"input": "수원에서 가볼만한 곳을 추천해주세요."},
    config={"configurable": {"session_id": "tourist_sqlite_1"}}
)

print("여행 가이드 답변:")
print(response.content)

In [ ]:
# SQLite에 저장된 대화 히스토리 확인
history = get_sqlite_history("tourist_sqlite_1")
pprint(history.messages)

In [ ]:
# 후속 질문 - SQLite에서 이전 대화를 불러옴
response = chain_with_sqlite.invoke(
    {"input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"},
    config={"configurable": {"session_id": "tourist_sqlite_1"}}
)

print("여행 가이드 답변:")
print(response.content)

> **참고**: `RunnableWithMessageHistory`는 LangChain 1.0에서도 여전히 동작하지만, 새로운 프로젝트에서는 `create_agent` + `checkpointer` 방식을 권장합니다. 이 섹션에서는 레거시 방식의 내부 동작 원리를 이해하기 위해 학습합니다.

### 2.3 메시지 트리밍 (TrimmedInMemoryHistory)

* 대화가 길어지면 컨텍스트 윈도우 제한에 도달할 수 있습니다. `TrimmedInMemoryHistory` 클래스는 메시지를 추가할 때 자동으로 오래된 메시지를 제거하여 최근 메시지만 유지합니다.

* `trim_messages` 함수를 활용하여 지정된 개수의 메시지만 유지하며, "last" 전략을 사용하여 가장 최근의 메시지부터 보존합니다.

* 이 방식은 메모리 효율성과 컨텍스트 품질의 균형을 맞추는 데 유용합니다.

In [ ]:
from langchain_core.messages import trim_messages

# 메시지 트리밍이 적용된 인메모리 히스토리 구현
class TrimmedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    """메시지 트리밍이 적용된 메모리 기반 히스토리"""
    messages: List[BaseMessage] = Field(default_factory=list)
    max_tokens: int = Field(default=4)  # 유지할 최대 메시지 수
    
    def __init__(self, max_tokens: int = 4, **kwargs):
        super().__init__(max_tokens=max_tokens, **kwargs)
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """메시지 추가 후 트리밍 수행"""
        self.messages.extend(messages)
        # 메시지 추가 후 트리밍 수행
        trimmer = trim_messages(
            strategy="last",
            max_tokens=self.max_tokens,
            token_counter=len  # 메시지 개수 기준
        )
        self.messages = trimmer.invoke(self.messages)
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
trimmed_store = {}

# 세션 ID로 트리밍된 히스토리 가져오기
def get_trimmed_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in trimmed_store:
        trimmed_store[session_id] = TrimmedInMemoryHistory(max_tokens=4)
    return trimmed_store[session_id]

print("TrimmedInMemoryHistory 클래스가 정의되었습니다.")

In [ ]:
# 트리밍된 히스토리를 사용하는 체인 구성
chain_with_trimmed = RunnableWithMessageHistory(
    chain_legacy,
    get_trimmed_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 첫 번째 질문
response = chain_with_trimmed.invoke(
    {"input": "서울에서 가볼만한 곳을 추천해주세요."},
    config={"configurable": {"session_id": "tourist_trimmed_1"}}
)

print("여행 가이드 답변:")
print(response.content)

In [ ]:
# 두 번째 질문
response = chain_with_trimmed.invoke(
    {"input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"},
    config={"configurable": {"session_id": "tourist_trimmed_1"}}
)

print("여행 가이드 답변:")
print(response.content)
print(f"\n현재 메시지 수: {len(get_trimmed_history('tourist_trimmed_1').messages)}")

In [ ]:
# 세 번째 질문 - 트리밍 효과 확인 (max_tokens=4이므로 오래된 메시지 제거됨)
response = chain_with_trimmed.invoke(
    {"input": "그 장소의 입장료는 얼마인가요?"},
    config={"configurable": {"session_id": "tourist_trimmed_1"}}
)

print("여행 가이드 답변:")
print(response.content)

# 트리밍된 히스토리 확인
print("\n[트리밍된 대화 히스토리]")
history = get_trimmed_history("tourist_trimmed_1")
for i, msg in enumerate(history.messages):
    role = msg.__class__.__name__.replace("Message", "")
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"{i+1}. [{role}]: {content}")

### 2.4 대화 요약 저장 (SummarizedInMemoryHistory)

* 대화가 길어질 경우, 전체 대화 내용을 요약하여 컨텍스트로 활용하는 방식으로 이전 대화의 핵심을 추출합니다.

* 메시지 히스토리가 지정된 길이(예: 6개의 메시지)를 초과할 경우, 이전 대화들을 요약하고 가장 최근의 메시지만 유지하는 방식으로 새로운 대화 히스토리를 구성합니다.

* 이러한 요약 메모리 방식을 통해 토큰 사용량을 크게 줄이면서도 대화의 핵심 문맥을 유지할 수 있으며, 특히 장시간 진행되는 대화에서 효과적입니다.

In [ ]:
from langchain_core.messages import SystemMessage

class SummarizedInMemoryHistory(BaseChatMessageHistory, BaseModel):
    """대화 요약이 적용된 메모리 기반 히스토리"""
    messages: List[BaseMessage] = Field(default_factory=list)
    summary_threshold: int = Field(default=6)  # 요약을 시작할 메시지 수
    llm: ChatOpenAI = Field(default_factory=lambda: ChatOpenAI(
        model="gpt-4.1-mini", temperature=0.1, top_p=0.9
    ))
    
    def add_messages(self, new_messages: List[BaseMessage]) -> None:
        """메시지 추가 및 필요시 요약 수행"""
        self.messages.extend(new_messages)
        
        print(f"현재 메시지 수: {len(self.messages)}")
        
        # 메시지 수가 임계값을 넘으면 요약 수행
        if len(self.messages) >= self.summary_threshold:
            # 마지막 사용자/AI 메시지 쌍 저장
            last_user_message = self.messages[-2]
            last_ai_message = self.messages[-1]
            
            # 요약 생성
            summary_prompt = (
                "Distill the above chat messages into a single summary message. "
                "Include as many specific details as you can. "
                "Use the original language and tone of the conversation."
            )
            
            summary_chain_messages = [
                SystemMessage(content=(
                    "You are a helpful assistant. "
                    "Your task is to summarize the conversation accurately."
                )),
                *self.messages[:-2],  # 마지막 대화 턴을 제외한 모든 메시지
                HumanMessage(content=summary_prompt)
            ]
            
            # 요약 생성
            summary = self.llm.invoke(summary_chain_messages)
            
            # 메시지 리스트 초기화 후 요약과 마지막 메시지 추가
            self.messages = [
                summary,
                last_user_message,
                last_ai_message
            ]
            print("→ 대화가 요약되었습니다.")
    
    def clear(self) -> None:
        self.messages = []

# 세션 저장소
summarized_store = {}

# 세션 ID로 요약된 히스토리 가져오기
def get_summarized_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in summarized_store:
        summarized_store[session_id] = SummarizedInMemoryHistory(summary_threshold=6)
    return summarized_store[session_id]

print("SummarizedInMemoryHistory 클래스가 정의되었습니다.")

In [ ]:
# 요약된 히스토리를 사용하는 체인 구성
chain_with_summarized = RunnableWithMessageHistory(
    chain_legacy,
    get_summarized_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 첫 번째 질문
response = chain_with_summarized.invoke(
    {"input": "서울에서 가볼만한 곳을 추천해주세요."},
    config={"configurable": {"session_id": "tourist_summarized_1"}}
)

print("여행 가이드 답변:")
print(response.content)

In [ ]:
# 두 번째 질문
response = chain_with_summarized.invoke(
    {"input": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"},
    config={"configurable": {"session_id": "tourist_summarized_1"}}
)

print("여행 가이드 답변:")
print(response.content)

In [ ]:
# 세 번째 질문 - 요약이 트리거됨 (6개 메시지 초과)
response = chain_with_summarized.invoke(
    {"input": "부산에서도 추천해주세요."},
    config={"configurable": {"session_id": "tourist_summarized_1"}}
)

print("여행 가이드 답변:")
print(response.content)

# 요약된 히스토리 확인
print("\n[요약된 대화 히스토리]")
history = get_summarized_history("tourist_summarized_1")
for i, msg in enumerate(history.messages):
    role = msg.__class__.__name__.replace("Message", "")
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"{i+1}. [{role}]: {content}")

---

## 3. `checkpointer` 파라미터를 활용한 메모리 관리 (권장 방식)

- LangChain 1.0에서는 `create_agent`가 에이전트 생성의 표준 방식입니다. `checkpointer` 파라미터를 통해 대화 이력을 자동으로 관리할 수 있습니다.

    - **간결한 코드**: StateGraph를 직접 구성할 필요 없이 몇 줄로 에이전트 생성
    - **자동 메모리 관리**: checkpointer가 thread_id별로 대화 내용을 자동 저장/복원
    
- **다양한 저장소 지원**: InMemorySaver, SqliteSaver, PostgresSaver 등

    | Checkpointer | 패키지 | 용도 |
    |-------------|--------|------|
    | `InMemorySaver` | `langgraph` (내장) | 개발/테스트용, 휘발성 |
    | `SqliteSaver` | `langgraph-checkpoint-sqlite` | 로컬 영구 저장 |
    | `PostgresSaver` | `langgraph-checkpoint-postgres` | 프로덕션 환경 |

### 3.1 InMemorySaver (메모리 기반)

* `InMemorySaver`는 가장 간단한 형태의 checkpointer로, 메모리에 상태를 저장합니다.
* 개발 및 테스트 환경에서 빠르게 프로토타이핑할 때 유용합니다.
* 프로세스가 종료되면 저장된 데이터가 사라지므로 프로덕션 환경에는 적합하지 않습니다.

* **create_agent vs StateGraph 비교:**

    ```python
    # StateGraph 방식 (약 20줄)
    workflow = StateGraph(MessagesState)
    workflow.add_node("call_model", call_model)
    workflow.add_edge(START, "call_model")
    workflow.add_edge("call_model", END)
    app = workflow.compile(checkpointer=checkpointer)

    # create_agent 방식 (약 5줄)
    agent = create_agent(
        model="gpt-4.1-nano",
        system_prompt="당신은 여행 가이드입니다.",
        checkpointer=InMemorySaver()
    )
    ```

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# create_agent를 사용하여 간단하게 대화형 에이전트 생성
agent = create_agent(
    model="gpt-4.1-nano",  # 사용할 모델
    tools=[],  # 도구 없이 순수 대화형 에이전트
    system_prompt="당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요.",
    checkpointer=InMemorySaver(),  # 메모리 기반 checkpointer
)

print("create_agent 기반 챗봇이 준비되었습니다.")

In [ ]:
# thread_id를 사용하여 대화 세션 구분
config = {"configurable": {"thread_id": "tourist_1"}}

# 첫 번째 질문
response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울에서 가볼만한 곳을 추천해주세요."}]},
    config=config
)

print("여행 가이드 답변:")
print(response["messages"][-1].content)

In [ ]:
# 대화 히스토리 확인 (현재 상태의 모든 메시지)
print("현재 대화 히스토리:")
for msg in response["messages"]:
    msg.pretty_print()

In [ ]:
# 같은 thread_id로 후속 질문 - 이전 대화를 기억함
response = agent.invoke(
    {"messages": [{"role": "user", "content": "이전에 추천한 장소 중에서 가장 인기 있는 곳은 어디인가요?"}]},
    config=config
)

print("여행 가이드 답변:")
print(response["messages"][-1].content)

In [ ]:
# 대화 히스토리 확인
print(f"총 메시지 수: {len(response['messages'])}")
print("\n대화 히스토리:")
for msg in response["messages"]:
    msg.pretty_print()

In [ ]:
# 다른 thread_id로 새로운 대화 시작 - 이전 대화와 독립적
config_new = {"configurable": {"thread_id": "tourist_2"}}

response_new = agent.invoke(
    {"messages": [{"role": "user", "content": "부산에서 맛집을 추천해주세요."}]},
    config=config_new
)

print("[새로운 대화 세션]")
print("여행 가이드 답변:")
print(response_new["messages"][-1].content)

### 3.2 SqliteSaver (영구 저장소)

* `SqliteSaver`는 SQLite 데이터베이스를 사용하여 대화 상태를 영구적으로 저장합니다.
* 프로세스가 재시작되어도 이전 대화를 복원할 수 있습니다.
* 로컬 개발 환경이나 단일 서버 환경에서 사용하기 적합합니다.

* **설치:**
    ```bash
    uv add langgraph-checkpoint-sqlite
    ```

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

# 1. 'with' 문을 사용하여 체크포인터를 생성합니다.
with SqliteSaver.from_conn_string("chat_history.db") as sqlite_checkpointer:
    
    # 2. 'with' 블록 안에서 에이전트를 생성하고 사용해야 합니다.
    agent_sqlite = create_agent(
        model="gpt-4.1-nano",
        tools=[],
        system_prompt="당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요.",
        checkpointer=sqlite_checkpointer, # 이제 올바른 객체가 전달됩니다.
    )

    print("SQLite 기반 챗봇이 준비되었습니다.")
    
    # 3. 에이전트 실행 예시
    # config = {"configurable": {"thread_id": "user_123"}}
    # response = agent_sqlite.invoke({"messages": [("user", "서울의 명소는?")]}, config)
    # print(response)

# 'with' 블록을 벗어나면 DB 연결이 자동으로 안전하게 닫힙니다.

In [ ]:
# SQLite 기반 대화 시작
with SqliteSaver.from_conn_string("chat_history.db") as sqlite_checkpointer:

    agent_sqlite = create_agent(
        model="gpt-4.1-nano",
        tools=[],
        system_prompt="당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요.",
        checkpointer=sqlite_checkpointer, # 이제 올바른 객체가 전달됩니다.
    )

    config_sqlite = {"configurable": {"thread_id": "sqlite_tourist_1"}}

    response = agent_sqlite.invoke(
        {"messages": [{"role": "user", "content": "제주도에서 꼭 가봐야 할 곳을 추천해주세요."}]},
        config=config_sqlite
    )

    print("여행 가이드 답변:")
    print(response["messages"][-1].content)

In [ ]:
# 후속 질문 - SQLite에서 이전 대화를 불러옴
with SqliteSaver.from_conn_string("chat_history.db") as sqlite_checkpointer:

    agent_sqlite = create_agent(
        model="gpt-4.1-nano",
        tools=[],
        system_prompt="당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요.",
        checkpointer=sqlite_checkpointer, # 이제 올바른 객체가 전달됩니다.
    )

    config_sqlite = {"configurable": {"thread_id": "sqlite_tourist_1"}}

    response = agent_sqlite.invoke(
        {"messages": [{"role": "user", "content": "그 중에서 아이와 함께 가기 좋은 곳은 어디인가요?"}]},
        config=config_sqlite
    )

    print("여행 가이드 답변:")
    print(response["messages"][-1].content)

In [ ]:
# 이전 대화 내역 조회
with SqliteSaver.from_conn_string("chat_history.db") as sqlite_checkpointer:
    
    # 1. 에이전트 생성
    agent_sqlite = create_agent(
        model="gpt-4.1-nano",
        tools=[],
        system_prompt="당신은 여행 가이드입니다. 관광객에게 유용한 정보를 제공하세요.",
        checkpointer=sqlite_checkpointer, # 이제 올바른 객체가 전달됩니다.
    )

    # 2. 불러오고 싶은 특정 thread_id 설정
    # 이 ID가 같으면 DB에서 이전 상태(State)를 자동으로 찾아옵니다.
    config = {"configurable": {"thread_id": "sqlite_tourist_1"}}

    # 3. [선택 사항] 이전 대화 내용이 잘 들어있는지 확인하기
    current_state = agent_sqlite.get_state(config)
    if current_state.values:
        print("--- 이전 대화 기록 발견 ---")
        for message in current_state.values.get("messages", []):
            role = "Human" if message.type == "human" else "AI"
            print(f"[{role}]: {message.content[:50]}...")
    else:
        print("--- 새로운 대화 세션입니다 ---")

---
# **[실습]**

- 메시지 트리밍과 대화 요약 저장을 결합하여 메시지를 관리하는 기능을 구현합니다. 

- **힌트**
    1. `TrimmedInMemoryHistory`의 `add_messages` 메서드를 참고하세요
    2. 트리밍 전에 제거될 메시지들을 먼저 식별하세요
    3. 제거될 메시지들에 대해서만 요약을 생성하세요
    4. 요약 메시지는 `summarized_messages` 리스트에 추가하세요

- **구현 순서:**
    1. 새 메시지 추가
    2. 현재 메시지 수가 `max_tokens` 초과 확인
    3. 초과 시, 제거될 메시지 식별
    4. 제거될 메시지 요약 생성
    5. 요약을 `summarized_messages`에 추가
    6. 현재 메시지는 트리밍 적용

In [ ]:
class TrimmedAndSummarizedHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage]
    max_tokens: int  # 트리밍 기준
    summary_threshold: int  # 요약 기준
    llm: ChatOpenAI
    summarized_messages: List[BaseMessage]  # 요약된 메시지 저장용

2. 메시지 처리 로직
- 새 메시지 추가시 max_tokens 체크
- 트리밍 발생하면 제거될 메시지 식별
- 제거 예정 메시지들은 요약하여 summarized_messages에 저장
- 현재 메시지는 트리밍된 상태로 유지

3. 요약 프로세스
- 트리밍으로 제거될 메시지들만 선별
- 선별된 메시지들에 대해 summary_chain 실행
- 요약본을 시스템 메시지로 변환하여 저장

In [ ]:
# 여기에 코드를 작성하세요.